<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>

<font color='blue'>
<br />
Иван, доброго дня! рад приветствовать тебя опять) <br />
(а дальше шаболннная обящательная часть, как ты заметил по другим спринтам)
<br />    

<br />
Меня зовут Николай Шавлюго. <br />И на этом этапе твоего движения к новой профессии от меня будут комментарии по написанному коду. <br />Чтобы меньше "мусорить эфир" и чтобы максимально наглядно отделяться от основного кода, есть предложение условиться в некоторых обозначениях:<br /> <br />
<u><b>ТАКОЙ ШРИФТ</b></u> - всегда начало комментария <br />
<font color='green'>такой шрифт</font> - комментарии о том, что всё ОК <br/>
<font color='orange'>такой шрифт</font> - комментарии о том, что всё ОК по результату,<br> однако есть на что обратить внимание в плане применения техник, или есть способы сделать более короткий или быстрый код <br/>
<font color='red'>такой шрифт</font> - комментарии о том, что есть критичный момент, влияющий на бизнес-результат проекта.<br/>
</font><br /><br />
<font color='green'>Моей целью является не "уличить" в не знании, а просто высказать сверху твоих знаний - свой опыт, что бы ты мог использовать его для своего дальнейшего успеха) И очень здорово будет, если тебе удастся задавать вопросы, да и вообще - всячески доставать меня, если я по каким-то причинам не приму проект:) При этом, из своего опыта скажу, хорошо и важно, когда переписка ревьюера и студента - сохраняется на следующие проекты и даже на будущую практическую деятельность.<br>
<BR> В ПУТЬ!<br>

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## План работ
1 Загрузка и изучение защищаемых данных

2 Необходимые преобразования. Подготовка датасета для МО

3 Теоретическое обоснование метода защиты и алгоритм действий

4 Обучение и проверка модели на исходных данных. Расчет метрик.

5 Обучение и проверка модели на преобразованных данных. Расчет метрик.

6 Сравнение метрик и выводы.

## 1. Загрузка данных

Загрузим необходимые библиотеки:

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Загрузим данные:

In [3]:
data = pd.read_csv('/datasets/insurance.csv')

Получим необходимую для дальнейшей работы информацию о датасете:

размерность:

In [4]:
data.shape

(5000, 5)

случайные записи:

In [5]:
data.sample(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
104,0,24.0,33500.0,1,0
2466,0,27.0,56400.0,1,0
3203,1,20.0,39800.0,1,0
2031,0,30.0,20900.0,0,0
1534,0,32.0,41800.0,3,0


типы данных:

In [6]:
data.dtypes

Пол                    int64
Возраст              float64
Зарплата             float64
Члены семьи            int64
Страховые выплаты      int64
dtype: object

пустые значения:

In [7]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

наличие дубликатов:

In [8]:
data.duplicated().sum()

153

корреляцию между признаками:

In [9]:
corr_data=data.corr()
corr_data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='orange'>
Напомню, что корреляция, как инструмент анализа, подходит к данным, предстваленными в виде количественной шкалы (возраст и пол например). Если данные в разных шкалах: возраст и пол - то данная корреляция не подходит. тут уже используется интсрумент "проверка гипотез" - сравнение групп из третьего спринта.<br>



удалим дубликаты:

In [15]:
data = data.drop_duplicates().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
Пол                  4847 non-null int64
Возраст              4847 non-null float64
Зарплата             4847 non-null float64
Члены семьи          4847 non-null int64
Страховые выплаты    4847 non-null int64
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


Преобразуем данные о возрасте и зарплате в целочисленные:

In [16]:
data['Возраст'] = data['Возраст'].astype('int64')
data['Зарплата'] = data['Зарплата'].astype('int64')

Подготовим тренировочную и обучающую выборки:

In [17]:
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=12345)
x_train.shape,y_train.shape, x_test.shape,y_test.shape

((3877, 4), (3877,), (970, 4), (970,))

### Выводы

данные страховой компании представляют собой  5000 записей 4х значений зависимых признаков и одного целевого.

Пропущенные значения отсутствуют.

Имеются дубликаты.
Значение коэффициентов корреляции между признаками указывает на наличие взаимосвязи возраста клиента с количеством страховых выплат.

Для дальнейшей работы были удалены дубликаты, а так же значения признаков 'Возраст'и 'Зарплата' были преобразованы в целочисленные значения.

Данные подготовлены для обучения и тестирования модели.


<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ОК, есть данные и хорошее аналитическое начало)

## 2. Умножение матриц

Показать, что умножение признаков на обратимую матрицу не повлияет на качество линейной регрессии

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)


**Обоснование**

Предсказания для линейной регрессионной модели находятся путем  умножения матрицы весов на  матрицу признаков:
$$
a=Xw
$$
измененная матрица признаков находится как:
$$
X_{enc}=(X P) 
$$
матрица весов линейной регрессии, обученной на преобразованных данных равна:
$$
w_{enc} = P^{-1}w
$$

Получаем, что расчитанные предсказания на изменненных данных: $$a_{enc}$$
равны:
$$
a_{enc} = X_{enc}w_{enc} = (X P) (P^{-1}w) = X w = a
$$
Качество предсказания на измененных данных не изменится.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
понятно)

## 3. Алгоритм преобразования

**Алгоритм**

1) Генерируем квадратную матрицу P с размерностью равной количеству признаков  из набора случайных чисел.

2) Проводим проверку P на существование обратной матрицы.

3) В случае отрицательного результата возвращаемся к пункту 1

3) Умножаем матрицы признаков на полученную квадратную матрицу и получаем измененные данные.

4) Обучаем модель линейной регрессии  на тренировочной выборке и получаем предсказания на тестовой для исходных и преобразованных данных

5) Находим значения метрик качества моделей.

**Обоснование**

Для преобразования признаков берется квадратная обратимая матрица - $P$, где $XP=X_{enc}$ - процесс сокрытия истинных данных , а $X_{enc}P^{-1}=X$. Ранее (Раздел 2) была обоснована возможность использования обратимой матрицы $P$ для изменения исходной матрицы признаков. 

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ОК.

## 4. Проверка алгоритма

Генерируем квадратную матрицу

In [18]:
np.random.seed(12345)
P = np.random.randint(100, size=(4, 4))
P

array([[98, 29,  1, 36],
       [41, 34, 29,  1],
       [59, 14, 91, 80],
       [73, 11, 77, 10]])

Проверим, обратима ли матрица P

In [19]:
P_inv = np.linalg.inv(P)
P_inv

array([[ 0.01003515, -0.0095093 , -0.00566525,  0.01014639],
       [-0.00416205,  0.03739133,  0.00370665, -0.01840899],
       [-0.0094485 ,  0.0042389 ,  0.00323802,  0.0076865 ],
       [ 0.0040751 , -0.00435212,  0.0123462 , -0.01300478]])

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ОТЛИЧНО! есть проверка на обратимость)

посмотрим на результат умножения матрицы на обратную:

In [20]:
result=P@P_inv
result

array([[ 1.00000000e+00,  1.52655666e-16, -3.46944695e-17,
         6.93889390e-17],
       [-9.54097912e-18,  1.00000000e+00, -2.60208521e-17,
         5.89805982e-17],
       [-9.71445147e-17,  5.55111512e-17,  1.00000000e+00,
         2.77555756e-16],
       [ 8.67361738e-18,  4.85722573e-17,  1.07552856e-16,
         1.00000000e+00]])

подготовим "шифрованные" данные:

In [21]:
x_train_enc=x_train@P
x_test_enc=x_test@P

Обучим модель линейной регрессии и вычислим метрики качества  на исходных данных:

In [22]:
model = LinearRegression()
model.fit(x_train, y_train)
predicted = pd.Series(model.predict(x_test))

print('R2 :',r2_score(y_test, predicted),'MSE:',mean_squared_error(y_test, predicted))

R2 : 0.416054922982786 MSE: 0.12311255285665611


Обучим модель линейной регрессии и вычислим метрики качества  на преобразованных данных:

In [23]:
model = LinearRegression()
model.fit(x_train_enc, y_train)
predicted = pd.Series(model.predict(x_test_enc))

print('R2 :',r2_score(y_test, predicted),'MSE:',mean_squared_error(y_test, predicted))

R2 : 0.4160549229840569 MSE: 0.12311255285638817


Метрики качества наших моделей практически совпадают.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ИВАН,  и с линейной алгеброй ты справился)<br>
Всё, в общем-то, по делу, со смыслом. Доказательство в п.2 - понятно)<br>
Да и весь проект - на уровне.<br>
Не забыл про проверку на обратимость! ОК<br>
    
Если рассматривать передачу данных третьим лицам, то к преимуществам данного подхода можно отнести, в первую очередь, ощутимую экономию ресурсов по сравнению с некоторыми известными алгоритмами шифрования, в особенности это касается ассиметричных алгоритмов с достаточно длинными ключами (AES, RSA), которые требуют много процессорного времени.<br><br>
    
К главному недостатку подхода можно отнести увеличение сложности проверки на вырожденность преобразования с ростом количества признаков в выборке. В рассмотренном случае необходимо подобрать подходящую матрицу размера  4×4 . Однако, если признаки будут исчисляться тысячами, что в теории всегда надо положить, то расчет соответствующих определителей может быть крайне ресурсозатратным. Но и эту проблему можно будет решить)<br><br>
<font color='green'><font color='green'>
Спасибо) Успешного дальнейшего обучения

### Выводы
Изучены данные по выплатам страховой компании клиентам.

Для деперсонализации данных обоснован алгоритм их преобразования, путем умножения на обратимую матрицу, который не снижает качество предсказания целевого признака.

Алгоритм прошел проверку для линейной регрессионной модели.

Проверка модели на исходных и измененных данных показала, что метрики качества модели остаются неизменными.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [х]  Весь код выполняется без ошибок
- [х ]  Ячейки с кодом расположены в порядке исполнения
- [х ]  Выполнен шаг 1: данные загружены
- [х ]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [ х]  Указан правильный вариант ответа
    - [ х]  Вариант обоснован
- [х ]  Выполнен шаг 3: предложен алгоритм преобразования
    - [ х]  Алгоритм описан
    - [х ]  Алгоритм обоснован
- [ х]  Выполнен шаг 4: алгоритм проверен
    - [х ]  Алгоритм реализован
    - [ х]  Проведено сравнение качества моделей до и после преобразования